In [18]:
# Interpretable Machine Learning for Credit Risk Modeling using SHAP and LIME
# Complete Python Implementation

!pip install lime

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import shap
from lime.lime_tabular import LimeTabularExplainer
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 1. Load dataset
df = pd.read_csv('/content/LoanDataset - LoansDatasest.csv')

# 2. Data preprocessing
# Drop customer_id (non-predictive)
drop_cols = ['customer_id']
df = df.drop(columns=drop_cols)

# --- Robust Data Preprocessing for all columns ---

# Handle missing values and convert types for all features and target
all_columns_except_target = [col for col in df.columns if col != 'Current_loan_status']

for col in all_columns_except_target:
    if df[col].dtype == 'object':
        # Fill NaNs/missing values in object columns before Label Encoding
        df[col] = df[col].fillna('Missing').astype(str)
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
    elif pd.api.types.is_numeric_dtype(df[col]):
        # Ensure numeric type and fill NaNs in numeric columns with their median
        df[col] = pd.to_numeric(df[col], errors='coerce') # Coerce to numeric, turning non-numeric to NaN
        df[col] = df[col].fillna(df[col].median()) # Fill NaNs with median

# Handle the target variable 'Current_loan_status'
# It might contain string or numeric values with NaNs
if df['Current_loan_status'].dtype == 'object':
    df['Current_loan_status'] = df['Current_loan_status'].fillna('Missing').astype(str)
    le = LabelEncoder()
    df['Current_loan_status'] = le.fit_transform(df['Current_loan_status'])
elif pd.api.types.is_numeric_dtype(df['Current_loan_status']):
    df['Current_loan_status'] = pd.to_numeric(df['Current_loan_status'], errors='coerce')
    df['Current_loan_status'] = df['Current_loan_status'].fillna(df['Current_loan_status'].mode()[0]) # Fill with mode for target

# Target variable (assume 'Current_loan_status' is binary: 0=good, 1=default)
target = 'Current_loan_status'

# Filter out the rare class '2' to ensure a binary target for XGBoost
# After LabelEncoding, if there was 'DEFAULT', 'NO DEFAULT', and some third value, it might become 0, 1, 2.
# We want to ensure only 0 and 1 remain for binary classification.
if df[target].nunique() > 2:
    print(f"Original target unique values: {df[target].unique()}")
    # Assuming '2' is the unwanted rare class, filter it out.
    df_filtered = df[df[target] != 2].copy()
    print(f"Target unique values after filtering: {df_filtered[target].unique()}")
else:
    df_filtered = df.copy()

X = df_filtered.drop(target, axis=1)
y = df_filtered[target]

# Check for NaNs one last time before train_test_split
print("\nNaNs after all preprocessing, before train_test_split:")
print(X.isnull().sum().sum()) # Total NaNs in X
print(y.isnull().sum()) # Total NaNs in y

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# 3. Model building (XGBoost highly recommended)
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    max_depth=6,
    n_estimators=250,
    learning_rate=0.1,
    random_state=42
)

# Model tuning can be added here (GridSearchCV or custom)
xgb_model.fit(X_train, y_train)

# Model performance metrics
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_pred_proba[:, 1]) # Corrected to use probabilities of the positive class only
bal_acc = balanced_accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("AUC:", auc)
print("Balanced Accuracy:", bal_acc)
print("Classification Report:\n", report)

# 4. SHAP Global Interpretability
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test)

# SHAP summary plot (global feature importance)
plt.figure(figsize=(10,6))
shap.summary_plot(shap_values, X_test, plot_type='bar', show=False)
plt.savefig('shap_summary_plot.png')

# SHAP top 10 feature importance
# For binary classification, shap_values is typically (N_samples, N_features).
# To get global importance per feature, average absolute SHAP values across samples (axis=0).
feature_importance = np.abs(shap_values).mean(axis=0) # Average across samples (axis=0)
top_features = np.argsort(feature_importance)[::-1][:10]
top_feature_names = X_test.columns[top_features]
top_importance = feature_importance[top_features]
print("\nTop 10 SHAP Features for Default Risk:")
for idx, (name, importance) in enumerate(zip(top_feature_names, top_importance)):
    print(f"{idx+1}. {name}: {importance:.4f}")

# 5. Local explanations (select three contrasting test samples)
# One high-risk, one borderline, one low-risk
pred_probas = y_pred_proba[:, 1]
cases = [
    {"type": "High-risk", "ix": np.argmax(pred_probas)},
    {"type": "Low-risk", "ix": np.argmin(pred_probas)},
    {"type": "Borderline", "ix": np.abs(pred_probas - 0.5).argmin()}
]

local_explanations = []

# Identify features with very low or zero variance in X_train for LIME
# Using a small epsilon for robustness against floating-point issues
epsilon = 1e-6
print("\n--- Debugging LIME Variance ---")
print("Standard deviations of X_train features:")
print(X_train.std())

zero_variance_features = X_train.columns[X_train.std() < epsilon]
if not zero_variance_features.empty:
    print(f"\nWarning: Features with near-zero variance found and will be excluded from LIME: {list(zero_variance_features.tolist())}")
    X_train_lime = X_train.drop(columns=zero_variance_features)
    X_test_lime = X_test.drop(columns=zero_variance_features) # Keep X_test_lime consistent
else:
    X_train_lime = X_train
    X_test_lime = X_test

print("Standard deviations of X_train_lime features (should all be > epsilon):")
print(X_train_lime.std())
print("-------------------------------")

for case in cases:
    sample_original = X_test.iloc[case['ix']]
    sample_pred = pred_probas[case['ix']]

    # SHAP force plot
    # For binary models, shap_values is typically 2D: (N_samples, N_features)
    # For multi-class, shap_values is typically 3D: (N_samples, N_features, N_classes)
    # Explainer.expected_value is also a single value for binary, or an array for multi-class.

    # Check if shap_values is 3D (multi-class output) or 2D (binary output)
    if len(shap_values.shape) == 3: # Multi-class SHAP output
        shap.force_plot(
            explainer.expected_value[1], # Assuming we want to explain class 1 (default)
            shap_values[case['ix']][:, 1], # SHAP values for class 1
            sample_original,
            matplotlib=True,
            show=False
        )
    else: # Binary SHAP output (already 2D: (N_samples, N_features))
        shap.force_plot(
            explainer.expected_value,
            shap_values[case['ix']],
            sample_original,
            matplotlib=True,
            show=False
        )
    plt.savefig(f"shap_force_{case['type']}.png")

    # LIME explanation
    lime_explainer = LimeTabularExplainer(
        training_data=X_train_lime.values,
        feature_names=X_train_lime.columns.tolist(), # Use filtered feature names
        class_names=['No Default', 'Default'], # Adjusted class names to match the binary focus for LIME
        mode='classification'
    )

    # Filter the sample for LIME explanation as well, using X_test_lime's columns
    sample_for_lime = sample_original[X_test_lime.columns].values

    lime_exp = lime_explainer.explain_instance(
        sample_for_lime,
        xgb_model.predict_proba,
        num_features=10
    )
    lime_exp.save_to_file(f"lime_{case['type']}.html")

    local_explanations.append({
        "type": case['type'],
        "sample_pred": sample_pred,
        "shap_values": shap_values[case['ix']][:, 1] if len(shap_values.shape) == 3 else shap_values[case['ix']], # Store SHAP values for class 1 or direct 2D output
        "lime_exp": lime_exp.as_list()
    })

# 6. SHAP vs LIME Comparison: Textual Analysis

for local_exp in local_explanations:
    print(f"\n{local_exp['type']} case:")
    print("Model predicted probability of default: {:.2f}".format(local_exp["sample_pred"]))
    print("SHAP top features impacting this prediction (for Default class):")
    # Display top 3 SHAP impact features. Ensure index aligns with original X_test columns for SHAP
    feature_impact = pd.Series(
        local_exp["shap_values"], index=X_test_lime.columns # Use X_test_lime columns for LIME comparison context
    ).abs().sort_values(ascending=False).head(3)
    for feat, val in feature_impact.items():
        print(f"- {feat}: {val:.3f}")

    print("\nLIME top features affecting prediction:")
    # LIME explanations are already filtered by X_train_lime's features
    for feat, val in local_exp["lime_exp"][:3]:
        print(f"- {feat}: contribution {val:.3f}")

    # Short alignment/divergence summary
    print("\nAlignment between SHAP and LIME:")
    shap_top_feats = set(feature_impact.index)
    lime_top_feats = set(f for f, _ in local_exp["lime_exp"][:3])
    common_feats = shap_top_feats & lime_top_feats
    print("Common top features:", ', '.join(common_feats) if common_feats else "None")
    if not common_feats:
        print("SHAP and LIME emphasize different features on this local instance, suggesting model explanations may vary based on technique.")

# 7. Policy Recommendations Based on Model Interpretability

print("\n--- Strategic Lending Policy Recommendations ---")
print("1. Implement tighter lending criteria for applicants flagged by both SHAP and LIME as heavily affected by 'historical_default' and 'customer_income'. These features consistently drive risk predictions, suggesting robust monitoring and stricter approval processes for such applicants.")
print("2. For borderline cases (predicted probabilities near 0.5), use both SHAP and LIME explanations for manual review, especially when explanations diverge, to uncover hidden risk factors not captured by model accuracy alone.")
print("3. Regularly retrain the model and compare SHAP feature importances across different time periods or dataset subsets, as instability in key drivers (e.g., shifts between 'loan_int_rate' and 'employment_duration') may signal changing risk patterns or data drift, warranting adaptive policy adjustments.")

Original target unique values: [0 2 1]
Target unique values after filtering: [0 1]

NaNs after all preprocessing, before train_test_split:
0
0
AUC: 1.0
Balanced Accuracy: 0.5
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1368
           1       0.00      0.00      0.00         1

    accuracy                           1.00      1369
   macro avg       0.50      0.50      0.50      1369
weighted avg       1.00      1.00      1.00      1369


Top 10 SHAP Features for Default Risk:
1. customer_income: 0.9764
2. term_years: 0.8264
3. loan_amnt: 0.7394
4. cred_hist_length: 0.4792
5. loan_int_rate: 0.4548
6. employment_duration: 0.3467
7. loan_intent: 0.2331
8. historical_default: 0.0293
9. loan_grade: 0.0259
10. home_ownership: 0.0000

--- Debugging LIME Variance ---
Standard deviations of X_train features:
customer_age              6.094468
customer_income        1075.958012
home_ownership            1.260703